In [45]:
%matplotlib inline

import pandas as pd

import matplotlib.pyplot as plt

import numpy as np

plt.style.use('ggplot')

pd.__version__

u'0.17.0'

In [46]:
SSF_subsidies = pd.read_csv("SSF_subsidies.csv")
countries = pd.read_csv("Country_Data.csv")
catch = pd.read_csv("SAU_Catches.csv")
type_EEZ = pd.read_csv("Subtype_Fleet_EEZ.csv")
all_subsidies = pd.read_csv("Subsidies2009.csv")

In [47]:
SSF_subsidies.columns.values

array(['ID', 'Cnumber', 'RegionID', 'RegionName', 'Type', 'SubType',
       'Category', 'ReEst_Subsidy2009', 'NewData', 'Developed', 'HDI_2005',
       'Countries', 'SSFsubsidies_assessed_percent',
       'weighted_by_sectorsize', 'Data_group',
       'SSF_totalCatch_EEZ_percent', '%SSF_totalCatch_Country',
       'SSF_USD_1000', 'Comment', 'DocSource', 'SourceVisitDate', 'Ref',
       'URL', 'User', 'OldComment'], dtype=object)

In [89]:
#  merge original all subsidies data with to align with subsidy 
#  type and whether the EEZ catch data or the FLeet catch data apply, 
#  see type_EEZ data, this is in preparation to later merge with SAU_catch data 

subsidies_type = pd.merge(
    left=all_subsidies,
    right=type_EEZ,
    how='left'
)


#  now this needs to be merged with country data to have coordinates per country
#  and have each country data point assigned to a subregion (thanks to Andres data)

subsidies_type_countries = pd.merge(
    left=subsidies_type,
    right=countries,
    how='left'
)


#  merge with SSF catch from SAU_catch data

subsidies_type_countries_catch = pd.merge(
    left=subsidies_type_countries,
    right=catch, how='left'
)

#  calculating mean SSF catch per subregion, 
#  needed to use to fill gaps for SSF subsidies that have not been assessed

catch_countries = pd.merge(
    left=catch, 
    right=countries, 
    how='left'
)
catch_by_subregion = catch_countries.groupby(['Subregion']).sum()
catch_by_subregion['mean_catch'] = (
    catch_by_subregion['SSF_Catch']/catch_by_subregion['Total_Catch']
)*100 
catch_by_subregion = catch_by_subregion.reset_index()
mean_catch = catch_by_subregion.loc[:,['Subregion','mean_catch']]


#  merging mean catch with all subsidies data
subsidies_type_countries_catch_meancatch = pd.merge(
    left=subsidies_type_countries_catch,right=mean_catch, how='left'
) 

#  preparing SSF subsidies by merging with country data
SSF_subsidies_countries = pd.merge(
    left=SSF_subsidies,right=countries, how='left'
)

#  from SSF subsidies assessed calculate mean per subregion and per type 
#  to fill gaps of countries that hvae not been assessed
SSF_by_subregion = SSF_subsidies_countries.groupby(
    ['Subregion','Type']).sum(
)


In [ ]:

SSF_by_subregion['mean_subsidies_subregion'] = (
    SSF_by_subregion['SSF_USD_1000']/SSF_by_subregion['ReEst_Subsidy2009']
)*100
SSF_by_subregion.to_csv('SSF_by_subregion.csv')

In [ ]:
SSF_by_subregion = SSF_by_subregion.reset_index()
mean_SSFsubsidies_subregion = SSF_by_subregion.loc[:,['Subregion','Type','mean_subsidies_subregion']]



#  as not all subsidies types are covered by countires assessed, 
#  mean values for region are being calculated to replace the empty cells
SSF_by_region = SSF_subsidies_countries.groupby(
    ['RegionName','Type']).sum(
)
SSF_by_region['mean_subsidies_region'] = (
    SSF_by_region['SSF_USD_1000']/SSF_by_region['ReEst_Subsidy2009']
)*100
SSF_by_region = SSF_by_region.reset_index()
mean_SSFsubsidies_region = SSF_by_region.loc[
    :,['RegionName','Type','mean_subsidies_region']
]

#  merge mean subsidies by type and subregion to all subsidies data
SSF_meancatch_meansubsidies_subregion = pd.merge(
    left=subsidies_type_countries_catch_meancatch,right=mean_SSFsubsidies_subregion, how='left'
)

#  merge mean subsidies by type and region to all subsidies data that now have included mean subsidies per subregion
SSF_meancatch_meansubsidies_region = pd.merge(
    left=SSF_meancatch_meansubsidies_subregion,right=mean_SSFsubsidies_region, how='left'
)

#  work in calc using this SSF_meancatch_meansubsidies_region csv file
#  fill gaps from mean subsidies per subregion with mean subsidies per region
#  create SSF_meancatch_meansubsidies_all.csv and upload as the new working data spreadsheet with all SSF subsidies
SSF_meancatch_meansubsidies_region.to_csv('SSF_meancatch_meansubsidies_region.csv')
SSF_meancatch_meansubsidies_all = pd.read_csv("SSF_meancatch_meansubsidies_all.csv")



In [49]:
np.sum(SSF_meancatch_meansubsidies_all['ReEst_Subsidy2009_original'])

35371111.003235795

In [83]:
#  create a new column 'Factor' and calculate as adjustment value for mean subsidies per subregion and mean SSF catch per subregion 
#  used to fill gaps for country that have not been assessed, this is done per subsidiey subtype also

SSF_meancatch_meansubsidies_all['Factor'] = SSF_meancatch_meansubsidies_all[
    'SSF_Catch_percent']/SSF_meancatch_meansubsidies_all['mean_catch'
                                                        ]

#  multiply the "Factor" value with the mean subsidy value per subregion
#  to estimate the value for SSF subsidies per subsidy type for the countries that have not been assessed
SSF_meancatch_meansubsidies_all['SSF_subsidies_estimates'] = SSF_meancatch_meansubsidies_all[
    'Factor']*SSF_meancatch_meansubsidies_all['mean_subsidies_subregion'
                                             ]
 
SSF_subsidies_short = SSF_subsidies_countries.loc[:,[
        'Countries','Type','SSFsubsidies_assessed_percent','weighted_by_sectorsize','Data_group','SSF_USD_1000']
                                                 ]

#  merge estimated values for subsidies with SSF assessed subsidies data
SSF_subsidies_estimates_assessed = pd.merge(
    left=SSF_meancatch_meansubsidies_all,right=SSF_subsidies_short,how='left'
)

#  use fillna function to use estimated values for SSF subsidies to fill gaps of countries that have not been assessed
SSF_subsidies_estimates_assessed['SSF_subsidies_percent'] = SSF_subsidies_estimates_assessed[
    'SSFsubsidies_assessed_percent'].fillna(
    SSF_subsidies_estimates_assessed['SSF_subsidies_estimates']
)

#  calculate USD values for all SSF for all subtypes and countries form the estimated and assessed percentages
SSF_subsidies_estimates_assessed['SSF_subsidies_USD'] = (
    SSF_subsidies_estimates_assessed['ReEst_Subsidy2009_original']/100
)*SSF_subsidies_estimates_assessed['SSF_subsidies_percent']

np.sum(SSF_subsidies_estimates_assessed['SSF_subsidies_USD'])


5170050.0764742009

In [62]:
#  save as csv file to add fuel subsidies
SSF_subsidies_estimates_assessed.to_csv('SSF_subsidies_estimates_assessed.csv')
#  read the newly created csv file now including fuel subsidy estimates
SSF_estimates_all = pd.read_csv("SSF_subsidies_withfuel.csv")

In [63]:
np.sum(SSF_estimates_all['SSF_subsidies_USD'])

7289241.7606364861

In [68]:
subsidies_category = SSF_estimates_all.groupby(['Category']).sum()
subsidies_category.columns.values

array(['ID', 'Cnumber', 'RegionID', 'ReEst_Subsidy2009_original',
       'NewData', 'HDI_2005', 'EEZ', 'CID', 'CLon', 'CLat', 'SIDS',
       'TotPop', 'HDI', 'GDP', 'Recreational', 'Subsistence', 'Artisanal',
       'Industrial', 'Total_Catch', 'SSF_Catch', 'SSF_Catch_percent',
       'mean_catch', 'mean_subsidies_subregion', 'Factor',
       'SSF_subsidies_estimates', 'SSFsubsidies_assessed_percent',
       'Data_group', 'SSF_USD_1000', 'SSF_subsidies_percent',
       'SSF_subsidies_USD'], dtype=object)

In [ ]:
subsidies_categoryloc[:,[
        'Category','ReEst_Subsidy2009_original','SSF_USD_1000']

In [ ]:
Subsidies_per_Category['percent SSF Category'] = Subsidies_per_Category['SSF subsidies']/Subsidies_per_Category['total subsidies']*100

In [79]:
 SSF_meancatch_meansubsidies_all

,Unnamed: 0,ID,Cnumber,RegionID,RegionName,Countries,Type,SubType,Category,ReEst_Subsidy2009_original,...,Subsistence,Artisanal,Industrial,Total_Catch,SSF_Catch,SSF_Catch_percent,mean_catch,mean_subsidies_subregion,Factor,SSF_subsidies_estimates
0,0,1,8,1,Europe,Albania,A1,Fisheries mangt.,Beneficial,387.260735,...,0.0,0.4,4.5,4.9,0.4,8.163265,14.823050,20.578290,0.550714,11.332758
1,1,2,8,1,Europe,Albania,A2,Fishery R&D,Beneficial,198.557500,...,0.0,0.4,4.5,4.9,0.4,8.163265,14.823050,17.746410,0.550714,9.773201
2,2,3,8,1,Europe,Albania,A3,MPAs,Beneficial,58.941816,...,0.0,0.4,4.5,4.9,0.4,8.163265,14.823050,22.431260,0.550714,12.353215
3,3,4,8,1,Europe,Albania,B1,Boat constsruct. & renov.,Capacity-enhancing,0.000000,...,0.0,0.4,4.4,4.8,0.4,8.333333,14.823050,9.303053,0.562187,5.230060
4,4,5,8,1,Europe,Albania,B2,Fisheries dev. projects,Capacity-enhancing,472.011572,...,0.0,0.4,4.4,4.8,0.4,8.333333,14.823050,3.937748,0.562187,2.213753
5,5,6,8,1,Europe,Albania,B3,Fishing port develop.,Capacity-enhancing,107.025943,...,0.0,0.4,4.4,4.8,0.4,8.333333,14.823050,26.746193,0.562187,15.036375
6,6,7,8,1,Europe,Albania,B4,Markt. & storage infrast.,Capacity-enhancing,152.502950,...,0.0,0.4,4.4,4.8,0.4,8.333333,14.823050,7.567187,0.562187,4.254178
7,7,8,8,1,Europe,Albania,B5,Tax exemption,Capacity-enhancing,0.000000,...,0.0,0.4,4.4,4.8,0.4,8.333333,14.823050,0.000000,0.562187,0.000000
8,8,9,8,1,Europe,Albania,B6,Fishing access,Capacity-enhancing,0.000000,...,0.0,0.4,4.4,4.8,0.4,8.333333,14.823050,3.102013,0.562187,1.743913
9,9,10,8,1,Europe,Albania,B7,Fuel subsidies,Capacity-enhancing,650.337185,...,0.0,0.4,4.4,4.8,0.4,8.333333,14.823050,NaN,0.562187,NaN
